In [ ]:
%pwd

In [ ]:
import numpy as np
import pandas as pd
import pickle
import csv
import os
import torch
from torch_geometric.data import Data
from tqdm import tqdm
import networkx as nx
from glob import glob
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm
from scipy import sparse
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv, SAGEConv, SGConv
import matplotlib.pyplot as plt
from scipy import sparse
np.random.seed(42)
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,accuracy_score
from torch_geometric.nn import VGAE
from torch import nn
import torch_geometric
from torch_geometric.data import DataLoader
from sklearn import metrics
import random

In [ ]:
def get_graph(mat):
#     mat = sparse.load_npz(path)
#     print(mat.shape)
    mat_coo = mat.tocoo()
    mat_csr = mat_coo.tocsr()
    G_new = nx.Graph(mat_csr)
    return G_new
    

In [ ]:
def load_feature(path):
    with open(path, 'rb') as f:
        total_feature = pickle.load(f)
        return total_feature

In [ ]:
total_feature= load_feature(r'D:\android_gan_mal2020\code_total_feature')

In [ ]:
from __future__ import print_function
import numpy as np

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional,SimpleRNN,GRU,Reshape,TimeDistributed,concatenate,BatchNormalization
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Activation, Flatten, UpSampling1D,MaxPooling1D
# from tensorflow.keras.layers.convolutional import MaxPooling1D
# from tensorflow.keras.layers.advanced_activations import LeakyReLU
# from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import model_from_json
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.backend.tensorflow_backend import set_session
# from tensorflow.keras.backend.tensorflow_backend import clear_session
# from tensorflow.keras.backend.tensorflow_backend import get_session  
from tensorflow.keras import metrics
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
# from tensorflow.keras.models import model_from_json

In [ ]:
# load json and create model
json_file = open(r"cnn98-12-nodiag.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
black_box_model = model_from_json(loaded_model_json)
# load weights into new model
black_box_model.load_weights(r"cnn98-12-nodiag.h5")

In [ ]:
class GraphSage(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GraphSage, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = SAGEConv(5, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.conv4 = SAGEConv(hidden_channels, hidden_channels)
        self.conv5 = SAGEConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
        x = x.relu()
        x = self.conv5(x, edge_index)
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        emb=x
        # 3. Apply a final classifier
        x = F.dropout(x, training=self.training)
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)   
        x = self.lin(x)
        return emb,x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
black_box_emb_model = GraphSage(hidden_channels=32)
optimizer_bb = torch.optim.Adam(black_box_emb_model.parameters(), lr=0.01)
criterion_bb = torch.nn.CrossEntropyLoss()
black_box_emb_model.load_state_dict(torch.load(r"GraphSagemal2020full_94acc.pt"))
black_box_emb_model = black_box_emb_model.to(device)

In [ ]:
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = SAGEConv(in_channels, out_channels)
        self.conv5 = SAGEConv(out_channels, out_channels)
        self.conv_mu = SAGEConv(out_channels, out_channels)
        self.conv_logstd = SAGEConv(out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv5(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
vae_generator = VGAE(VariationalGCNEncoder(5,32))
optimizer_vae_generator = torch.optim.Adam(vae_generator.parameters(), lr=0.01)
# vae_generator.load_state_dict(torch.load(r"vgae_mal_final_no_diag.pt"))
vae_generator = vae_generator.to(device)

In [ ]:
class GraphSageDis(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GraphSageDis, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = SAGEConv(5, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.conv4 = SAGEConv(hidden_channels, hidden_channels)
        self.conv5 = SAGEConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
        x = x.relu()
        x = self.conv5(x, edge_index)
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        emb=x
        # 3. Apply a final classifier
        x = F.dropout(x, training=self.training)
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)   
        x = self.lin(x)
        return emb,x

In [ ]:
discriminator_model = GraphSageDis(hidden_channels=32)
optimizer_discriminator = torch.optim.Adam(discriminator_model.parameters(), lr=0.01)
criterion_discriminator = torch.nn.CrossEntropyLoss()
discriminator_model = discriminator_model.to(device)

In [ ]:
class MyEnsemble(torch.nn.Module):
    def __init__(self, vae_generator, discriminator):
        super(MyEnsemble, self).__init__()
        self.vae_generator = vae_generator
        self.discriminator = discriminator
        
    def forward(self, x, edge_index,batch,full,permission):
        z = self.vae_generator.encode(x,edge_index)
#         print(z)
        adj=torch.exp(-torch.cdist(z,z, p=2.0))
        adj = (adj>0.98).float()*1
#         print(adj)
        #to remove self loop
        generated_edge_index_sparse=torch_geometric.utils.dense_to_sparse(adj)[0]    
         # self loop remove
        generated_edge_index_sparse=torch_geometric.utils.remove_self_loops(generated_edge_index_sparse)[0]
        try:
        # convert back to adjacency matrix to perform and and or operation
            generated_edge_index_dense=torch_geometric.utils.to_dense_adj(generated_edge_index_sparse,max_num_nodes=1000)[0]
        except:
            generated_edge_index_dense=torch_geometric.utils.to_dense_adj(edge_index,max_num_nodes=1000)[0]
        #perform and operation with full graph
        generated_edge_index_valid_links=torch.logical_and(full,generated_edge_index_dense)*1
        #perform or operation with the original edge_index to preserver the malware characteristic
        edge_index_dense=torch_geometric.utils.to_dense_adj(edge_index,max_num_nodes=1000)[0]
        generated_edge_index_final=torch.logical_or(generated_edge_index_valid_links,edge_index_dense)*1
        #cover it into sparse to pass into discriminator
        generated_edge_index_final=torch_geometric.utils.dense_to_sparse(generated_edge_index_final)[0]
        emb,output = self.discriminator(x,generated_edge_index_final,batch)
        return output,z


In [ ]:
generator=MyEnsemble(vae_generator, discriminator_model)
generator=generator.to(device)
print(generator)

In [ ]:
# overall_graph_path=r'code_block_id_tr_reduced_tr.npz'
# sparse_overall_graph = sparse.load_npz(overall_graph_path)
# overall_graph_edge_index_coo=sparse_overall_graph.tocoo()
# overall_graph_edge_index = torch.tensor([overall_graph_edge_index_coo.row.tolist(),overall_graph_edge_index_coo.col.tolist()], dtype=torch.long) 
# pos_overall_graph_edge_index = overall_graph_edge_index.to(device)
# pos_overall_graph_edge_index =torch_geometric.utils.remove_self_loops(pos_overall_graph_edge_index)[0]
# pos_overall_adj=torch_geometric.utils.to_dense_adj(pos_overall_graph_edge_index)[0]
# print(pos_overall_adj)

In [ ]:
overall_graph=load_feature("overall_edge_list")
overall_graph_edge_index = torch.tensor([overall_graph[0],overall_graph[1]], dtype=torch.long) 
pos_overall_graph_edge_index = overall_graph_edge_index.to(device)
pos_overall_graph_edge_index = torch_geometric.utils.remove_self_loops(pos_overall_graph_edge_index)[0]
pos_overall_adj=torch_geometric.utils.to_dense_adj(pos_overall_graph_edge_index)[0]
print(pos_overall_adj)

In [ ]:
perm_df=pd.read_csv(r'permission_reduced_mal2020.csv')

In [ ]:
perm_df.head(2)

In [ ]:
perm_df.iloc[:,1:6852]

In [ ]:
#loading dataset malware
class YooChooseDatasetBen(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseDatasetBen, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return [r'code_block.dataset']

    def download(self):
        pass
        
    def process(self):
        data_list = []
        edge_index_path=[r"class0"]
        current=0
        for i in edge_index_path: 
            edge_indexes = glob(f'{i}/*.npz')
            print(len(edge_indexes))
            for edge_index_path in edge_indexes:
                current=current+1
                print(current)
                y=[0]
                y = torch.tensor(y)
                try:
                    sparse_arr = sparse.load_npz(edge_index_path)
                    sparse_arr.setdiag(0)
                    sparse_arr.eliminate_zeros()
                    edge_index_processed=sparse_arr
                    edge_index = torch.tensor([edge_index_processed.row.tolist(),edge_index_processed.col.tolist()], dtype=torch.long)
                    x = torch.tensor(total_feature,dtype=torch.float)
                    app_name=os.path.basename(edge_index_path).replace('.npz','')
                    permission_data=perm_df.loc[perm_df['App']==app_name].iloc[:,1:6852].values.tolist()[0]
                    data = Data(x=x, edge_index=edge_index, y=y,permission=permission_data)
                    data_list.append(data)
                except Exception as e:
                    print(e)
                    print(edge_index_path)
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
custom_dataset_benign= YooChooseDatasetBen(root=r'bening_gan')

In [ ]:
torch.manual_seed(12345)
dataset_benign = custom_dataset_benign.shuffle()
train_dataset_benign = dataset_benign[:2586]
test_dataset_benign = dataset_benign[2586:]
print(test_dataset_benign)
print(f'Number of training graphs: {len(train_dataset_benign)}')
print(f'Number of test graphs: {len(test_dataset_benign)}')
print(dataset_benign.num_node_features)

In [ ]:
def discriminator_train(loader):
    discriminator_model.train()
    running_loss = 0.0
    correct = 0
    benign_count=0
    malware_count=0
    for data in loader:  # Iterate in batches over the training dataset.
         data.x = data.x.to(device)
         data.edge_index= data.edge_index.to(device)
         data.batch=data.batch.to(device)
         data.y = data.y.to(device)
         emb,out = discriminator_model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
#          print(out)
         pred = out.argmax(dim=1)
#          print(pred.dtype)
#          print(data.y.dtype)
         output=pred.cpu().numpy()
         for i in output:
            if i ==0:
                benign_count+=1
            else:
                malware_count+=1
         
         correct += int((pred == data.y).sum())
#          print(pred[0].dtype)
#          print(data.y[0].dtype)
#          print("prediction",pred)
#          print("data y",data.y)
         loss = criterion_discriminator(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer_discriminator.step()  # Update parameters based on gradients.
         optimizer_discriminator.zero_grad() # Clear gradients.
         running_loss += loss.item() * data.num_graphs
    running_loss /= len(loader.dataset)
    print(benign_count)
    print(malware_count)
    return correct / len(loader.dataset),running_loss

In [ ]:
def get_discriminator_prediction(data):
    discriminator_model.eval()
    data=data.to(device)
    out= discriminator_model(data.x, data.edge_index, data.batch)  
    pred = out.argmax(dim=1)
    pred_numpy = pred.cpu().numpy()
    return pred_numpy
    

In [ ]:
def mydecoder(edge_index,z,pos_overall_adj):
    adj=torch.exp(-torch.cdist(z,z, p=2.0))
    adj = (adj>0.98).float()*1
#     adj = torch.matmul(z, z.t())
#     adj=torch.sigmoid(adj)
    #to remove self loop
    generated_edge_index_sparse=torch_geometric.utils.dense_to_sparse(adj)[0] 
    
     # self loop remove
    generated_edge_index_sparse=torch_geometric.utils.remove_self_loops(generated_edge_index_sparse)[0]
    # convert back to adjacency matrix to perform and and or operation
    try:
        generated_edge_index_dense=torch_geometric.utils.to_dense_adj(generated_edge_index_sparse,max_num_nodes=1000)[0]
        exception=False
    except:
        generated_edge_index_dense=torch_geometric.utils.to_dense_adj(edge_index,max_num_nodes=1000)[0]
        exception=True
#     print(pos_overall_adj)
    #perform and operation with full graph
    generated_edge_index_valid_links=torch.logical_and(pos_overall_adj,generated_edge_index_dense)*1
    #print(generated_edge_index_valid_links)
    #perform or operation with the original edge_index to preserver the malware characteristic
    edge_index_dense=torch_geometric.utils.to_dense_adj(edge_index,max_num_nodes=1000)[0]
    generated_edge_index_final=torch.logical_or(generated_edge_index_valid_links,edge_index_dense)*1
    #cover it into sparse to pass into discriminator
    generated_edge_index_final=torch_geometric.utils.dense_to_sparse(generated_edge_index_final)[0]
    return generated_edge_index_final,exception
    

In [ ]:
def get_black_box_p_emb_prediction(permission_data,embedding):
    pred_soft=black_box_model.predict([np.array(permission_data).reshape(1,6851),embedding.detach().cpu().numpy()])
    if pred_soft[0]<0.5:
        pred=0
    else:
        pred=1
    return torch.tensor([pred])

In [ ]:
benign_list=[]
for data in train_dataset_benign:
    data.y= torch.tensor([0])
    benign_list.append(data)

In [ ]:
def true_positive_rate_calculator(prediction_list,true_list):
    confusion=confusion_matrix(true_list,prediction_list)
    print(confusion)
    TP=confusion[1][1]
    FN=confusion[1][0]
    true_positive_rate=TP/(TP+FN)
    return true_positive_rate
    
    

In [ ]:
def get_embedding(data):
    discriminator_model.eval()
    data=data.to(device)
    emb,out = discriminator_model(data.x, data.edge_index, data.batch)  
    pred = out.argmax(dim=1)
    return pred,emb

In [ ]:
def get_embedding_bb(data):
    black_box_emb_model.eval()
    data=data.to(device)
    emb,out = black_box_emb_model(data.x, data.edge_index, data.batch)  
    pred = out.argmax(dim=1)
    return pred,emb

In [ ]:
malware_network_perm=[]
edge_index_path=[r"class1"]
current=0
exception_count=0
generated_malware_list=[]
generated_prediction=[]
for i in edge_index_path: 
    edge_indexes = glob(f'{i}/*.npz')
    for edge_index_path in edge_indexes:
        current=current+1
        #get app name
        app_name=os.path.basename(edge_index_path).replace('.npz','')
        #get permission
        permission_data=perm_df.loc[perm_df['App']==app_name].iloc[:,1:6852].values.tolist()[0]
        #load network from data from path
        sparse_arr = sparse.load_npz(edge_index_path)
        #transform intoo coo
        sparse_arr.setdiag(0)
        sparse_arr.eliminate_zeros()
        edge_index_processed=sparse_arr
        #convert in to edge index
        edge_index = torch.tensor([edge_index_processed.row.tolist(),edge_index_processed.col.tolist()], dtype=torch.long)
        malware_network_perm.append((edge_index,permission_data))
        if current==8474:
            break

In [ ]:
# print(malware_network_perm[0])

In [ ]:
count=0
true_positive_calcu=[]
#for batching will use data loader to batch and pass on to discriminator
for epoch in range(100):
    print("current iteration",epoch)
    generated_malware_list=[]
    exception_count=0
    generated_prediction=[]
    try:
        current=0
        exception_count=0
        generated_malware_list=[]
        generated_prediction=[]
        for edge_perm in malware_network_perm:
            current=current+1
            #get permission
            permission_data=edge_perm[1]
            edge_index = edge_perm[0]
            #get the x feature
            x = torch.tensor(total_feature,dtype=torch.float)
            random_data = random.choice(benign_list)
            edge_index_0=random_data.edge_index[0].split(int(len(random_data.edge_index[0])/2))[1]
            edge_index_1=random_data.edge_index[1].split(int(len(random_data.edge_index[1])/2))[1]
            benign_noise = torch.empty([2,int(len(random_data.edge_index[1])/2)],dtype=int)
            benign_noise[0]=edge_index_0
            benign_noise[1]=edge_index_1
            noise_edge_index=torch.cat((edge_index,benign_noise),1)
            malware = Data(x=x, edge_index=noise_edge_index)
            malware=malware.to(device)
            z = vae_generator.encode(malware.x,malware.edge_index)
            edge_index,exception=mydecoder(malware.edge_index,z,pos_overall_adj)
            if exception:
                exception_count+=1
            x = torch.tensor(total_feature,dtype=torch.float)
            data = Data(x=x, edge_index=edge_index)
            data.batch=torch.zeros(1000, dtype=torch.long)
            out_graph,emb=get_embedding_bb(data)
            out_perm_graph=get_black_box_p_emb_prediction(permission_data,emb)
            generated_prediction.append(out_perm_graph)
            data.edge_index.cpu()
            data.x=data.x.cpu()
            data.batch=data.batch.cpu()
            data.y=out_perm_graph
            data.batch=None
            generated_malware_list.append(data)
    except Exception as e:
        print(e)
    print("malware generation complete")
    print("graph generation",exception_count)
    pred_list_list=[l.tolist() for l in generated_prediction]
    predlist = [item for sublist in pred_list_list for item in sublist]
    # print(predlist)
    predlist.append(0)
    predlist.append(1)
    true_label_check=[1]*len(predlist)
    print("true positive rate after generation",true_positive_rate_calculator(predlist,true_label_check))
    tpr=true_positive_rate_calculator(predlist,true_label_check)
    if tpr<=0.60:
        print("saved here 0.60")
        torch.save(vae_generator.state_dict(), r"newvgaecnnbb_final_"+str(tpr)+".pt")
    if tpr<=0.50:
        print("saved here 0.50")
        torch.save(vae_generator.state_dict(), r"newvgaecnnbb_final_"+str(tpr)+".pt")
    if tpr<=0.40:
        print("saved here 40")
        torch.save(vae_generator.state_dict(), r"newvgaecnnbb_final_"+str(tpr)+".pt")
    #getting true positive rate after graph generation
    #combine_list=generated_malware_list+benign_list
    benign_train_loader = DataLoader(benign_list, batch_size=64, shuffle=True)
    malware_train_loader= DataLoader(generated_malware_list, batch_size=64, shuffle=True)
    for inner_epoch in range(3):
        train_acc_benign,train_loss_benign = discriminator_train(benign_train_loader)
        print("training benign discriminator")
        print(f'Epoch: {inner_epoch:03d}, discriminator Train Acc: {train_acc_benign:.4f},discriminator Train loss:{train_loss_benign:.4f}')
    for inner_epoch in range(10):
        train_acc_malware,train_loss_malware = discriminator_train(malware_train_loader)
        print("training malware discriminator")
        print(f'Epoch: {inner_epoch:03d}, discriminator Train Acc: {train_acc_malware:.4f},discriminator Train loss:{train_loss_malware:.4f}')
    #---------------------------------------------------------------------------
    # train generator
    for generator_epoch in range(3):
        running_loss = 0.0
        running_loss_dis=0.0
        current=0
        exception_count=0
        generated_malware_list=[]
        generated_prediction=[]
        for edge_perm in malware_network_perm:
            current=current+1
            #get permission
            permission_data=edge_perm[1]
            #load network from data from path
            #convert in to edge index
            edge_index = edge_perm[0]
            #get the x feature
            x = torch.tensor(total_feature,dtype=torch.float)
            random_data = random.choice(benign_list)
            edge_index_0=random_data.edge_index[0].split(int(len(random_data.edge_index[0])/2))[1]
            edge_index_1=random_data.edge_index[1].split(int(len(random_data.edge_index[1])/2))[1]
            benign_noise = torch.empty([2,int(len(random_data.edge_index[1])/2)],dtype=int)
            benign_noise[0]=edge_index_0
            benign_noise[1]=edge_index_1
            noise_edge_index=torch.cat((edge_index,benign_noise),1)
            data = Data(x=x, edge_index=noise_edge_index)
            data.batch=torch.zeros(1000, dtype=torch.long)
            data.y=torch.tensor([0])
            data=data.to(device)
            out,z=generator(data.x,data.edge_index,data.batch,pos_overall_adj,permission_data)
            out=out.to(device)
            loss = vae_generator.recon_loss(z,data.edge_index)
            loss_discriminator = criterion_discriminator(out, data.y)
#             loss=loss_discriminator
            loss = loss+(1 / data.num_nodes) * vae_generator.kl_loss()+loss_discriminator
            loss.backward()  # Derive gradients.
            optimizer_vae_generator.step()  # Update parameters based using gradients.
            optimizer_vae_generator.zero_grad() # Clear gradients.
            running_loss += loss 
            running_loss_dis += loss_discriminator 
            data.batch=None
            data.edge_index=data.edge_index.cpu()
            data.x=data.x.cpu()
            data.y=data.y.cpu()
        print("final_loss")
        running_loss /= len(edge_indexes)
        running_loss_dis /= len(edge_indexes)
        vgae_loss= running_loss - running_loss_dis
        print("loss of vgae",vgae_loss)
        print("loss of generator",running_loss)
        print('loss of discriminator',running_loss_dis)

In [ ]:
torch.nn.functional.binary_cross_entropy_with_logits(torch.tensor([0.0]),torch.tensor([0.1]),reduce=False)

In [ ]:
count=0
true_positive_calcu=[]
#for batching will use data loader to batch and pass on to discriminator
for epoch in range(100):
    print("current iteration",epoch)
    generated_malware_list=[]
    exception_count=0
    for malware in train_dataset_malware:
        
        try:
            random_data = random.choice(benign_list)
            edge_index_0=random_data.edge_index[0].split(int(len(random_data.edge_index[0])/2))[0]
            edge_index_1=random_data.edge_index[1].split(int(len(random_data.edge_index[1])/2))[0]
            benign_noise = torch.empty([2,int(len(random_data.edge_index[1])/2)],dtype=int)
            benign_noise[0]=edge_index_0
            benign_noise[1]=edge_index_1
            noise_edge_index=torch.cat((malware.edge_index,benign_noise),1)
            malware.edge_index=noise_edge_index
            count=count+1
            malware=malware.to(device)
            z = vae_generator.encode(malware.x,malware.edge_index)
            edge_index,exception=mydecoder(malware.edge_index,z,pos_overall_adj)
            if exception:
                exception_count+=1
            x = torch.tensor(total_feature,dtype=torch.float)
            data = Data(x=x, edge_index=edge_index)
            data.batch=torch.zeros(7017, dtype=torch.long)
            #get the black_box prediction
            out = get_black_box_prediction(data).cpu()
            data.edge_index=data.edge_index.cpu()
            data.edge_index.cpu()
            data.x=data.x.cpu()
            data.batch=data.batch.cpu()
            data.y=out.cpu()
            out = out.cpu().numpy()
            data.batch=None
            generated_malware_list.append(data)
        except Exception as e:
            print(e)
    print("malware generation complete")
    print("graph generation",exception_count)
    #combine_list=generated_malware_list+benign_list
    benign_train_loader = DataLoader(benign_list, batch_size=64, shuffle=True)
    malware_train_loader= DataLoader(generated_malware_list, batch_size=64, shuffle=True)
    train_acc_benign,train_loss_benign = discriminator_train(benign_train_loader)
    print("training benign discriminator")
    print(f'Epoch: {epoch:03d}, discriminator Train Acc: {train_acc_benign:.4f},discriminator Train loss:{train_loss_benign:.4f}')
    for inner_epoch in range(10):
        train_acc_malware,train_loss_malware = discriminator_train(malware_train_loader)
        print("training malware discriminator")
        print(f'Epoch: {inner_epoch:03d}, discriminator Train Acc: {train_acc_malware:.4f},discriminator Train loss:{train_loss_malware:.4f}')

    #---------------------------------------------------------------------------
    # train generator
    for generator_epoch in range(3):
        running_loss = 0.0
        running_loss_dis=0.0
        for data in train_dataset_malware:
            data.y=torch.tensor([0])
            data.batch=torch.zeros(7017, dtype=torch.long)
            random_data = random.choice(benign_list)
            edge_index_0=random_data.edge_index[0].split(int(len(random_data.edge_index[0])/2))[0]
            edge_index_1=random_data.edge_index[1].split(int(len(random_data.edge_index[1])/2))[0]
            benign_noise = torch.empty([2,int(len(random_data.edge_index[1])/2)],dtype=int)
            benign_noise[0]=edge_index_0
            benign_noise[1]=edge_index_1
            noise_edge_index=torch.cat((data.edge_index,benign_noise),1)
            data.edge_index=noise_edge_index
            data=data.to(device)

            out,z=generator(data.x,data.edge_index,data.batch,pos_overall_adj)
            loss = vae_generator.recon_loss(z,data.edge_index)
            loss_discriminator = criterion_discriminator(out, data.y)
    #        loss=loss_discriminator
    #         print(loss_discriminator)
            loss = loss+(1 / data.num_nodes) * vae_generator.kl_loss()+loss_discriminator
            loss.backward()  # Derive gradients.
            optimizer_vae_generator.step()  # Update parameters based on gradients.
            optimizer_vae_generator.zero_grad() # Clear gradients.
            running_loss += loss 
            running_loss_dis += loss_discriminator 
            data.batch=None
            data.edge_index=data.edge_index.cpu()
            data.x=data.x.cpu()
            data.y=data.y.cpu()
            pred = out.argmax(dim=1)
        print("final_loss")
        running_loss /= len(train_dataset_malware)
        running_loss_dis /= len(train_dataset_malware)
        print("loss of generator",running_loss)
        print('loss of discriminator',running_loss_dis)
    #getting tpr after training generator
    generated_malware_list=[]
    prediction=[]
    exception_count=0
    for malware in train_dataset_malware:
        random_data = random.choice(benign_list)
        edge_index_0=random_data.edge_index[0].split(int(len(random_data.edge_index[0])/2))[0]
        edge_index_1=random_data.edge_index[1].split(int(len(random_data.edge_index[1])/2))[0]
        benign_noise = torch.empty([2,int(len(random_data.edge_index[1])/2)],dtype=int)
        benign_noise[0]=edge_index_0
        benign_noise[1]=edge_index_1
        noise_edge_index=torch.cat((malware.edge_index,benign_noise),1)
        malware.edge_index=noise_edge_index
        malware=malware.to(device)
        z = vae_generator.encode(malware.x,malware.edge_index)
        edge_index,exception=mydecoder(malware.edge_index,z,pos_overall_adj)
        if exception:
            exception_count+=1
        x = torch.tensor(total_feature,dtype=torch.float)
        data = Data(x=x, edge_index=edge_index)
        data.batch=torch.zeros(7017, dtype=torch.long)
        #get the black_box prediction
        out = get_black_box_prediction(data).cpu()
        data.edge_index=data.edge_index.cpu()
        data.x=data.x.cpu()
        data.batch=data.batch.cpu()
        data.y=out.cpu()
        out = out.cpu().numpy()
        prediction.append(out)
        data.batch=None
        generated_malware_list.append(data)
    pred_list_list=[l.tolist() for l in prediction]
    predlist = [item for sublist in pred_list_list for item in sublist]
    predlist.append(0)
    predlist.append(1)
    # print(predlist)
    true_label_check=[1]*len(predlist)
    print("after generator graph generation exception",exception_count)
    tpr=true_positive_rate_calculator(predlist,true_label_check)
    print("true positive rate after training generator",tpr)
    true_positive_calcu.append(true_positive_rate_calculator(predlist,true_label_check))
    if tpr<=0.65:
        print("saved here 0.6")
        torch.save(vae_generator.state_dict(), r"D:\android_gan\vgae_final_"+str(tpr)+".pt")
    if tpr<=0.5:
        print("saved here 0.5")
        torch.save(vae_generator.state_dict(), r"D:\android_gan\vgae_final_"+str(tpr)+".pt")
    if tpr<=0.45:
        print("saved here 0.45")
        torch.save(vae_generator.state_dict(), r"D:\android_gan\vgae_final_"+str(tpr)+".pt")
    
    
    
    

    

